In [1]:
from dqn.policy import PolicyEpsilongGreedy
from dqn.dqn_base import DeepQLearning, DQNFactory, DQNType
from dqn.neural_net import NNLunarLander, NNLunarLanderDueling
from dqn.environment import DQNEnvironment
from dqn.hp_tuner import HPTuner
from dqn.epsilon import Epsilon
from ray import tune
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [2]:
device = "cpu"
hidden_size = 128
epsilon = Epsilon(eps_start=1.0, eps_end=0.15, eps_last_episode=100)

In [3]:
env = DQNEnvironment("LunarLander-v2")

Tune hyperparameters, learning rate and gamma

In [9]:
hp = HPTuner(
    50,
    1000,
    config={"lr": tune.loguniform(1e-4, 1e-3), "gamma": tune.loguniform(0.9, 1.0)},
)
best_hp = hp.tune(algo_type=DQNType.DQN, env=env.env, policy=PolicyEpsilongGreedy(device), q_net=NNLunarLander(hidden_size, env.observation_size(), env.number_of_actions()))
print(best_hp)

{'lr': 0.0006955077074830977, 'gamma': 0.9827600443940492}


Run the algorithm with the tuned hyperparameters

In [4]:
# !rm -rf lightning_logs/*
algo = DeepQLearning(env=env.env, policy=PolicyEpsilongGreedy(device), q_net=NNLunarLanderDueling(hidden_size, env.observation_size(), env.number_of_actions()), double_dqn=False)

/Users/meinczinger/.pyenv/versions/3.10.9/envs/rl/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'q_net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['q_net'])`.
  rank_zero_warn(
/Users/meinczinger/.pyenv/versions/3.10.9/envs/rl/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:249: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [5]:
match(device):
    case 'cpu':
        trainer = Trainer(
            max_epochs=10_000,
            callbacks=[EarlyStopping(monitor="episode/Return", mode="max", patience=500)],
        )
    case 'mps':
        trainer = Trainer(
            max_epochs=10_000,
            callbacks=[EarlyStopping(monitor="episode/Return", mode="max", patience=500)],
            accelerator="mps", devices=1
        )


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/meinczinger/.pyenv/versions/3.10.9/envs/rl/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:200: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [6]:
import tempfile
import shutil
from tensorboard import notebook
shutil.rmtree(tempfile.gettempdir()+"/.tensorboard-info", ignore_errors=True)

%load_ext tensorboard
%tensorboard --logdir lightning_logs --port 6007

notebook.display(port=6007, height=1000)

Selecting TensorBoard with logdir lightning_logs (started 0:00:00 ago; port 6007, pid 33355).


In [7]:
trainer.fit(algo)


  | Name         | Type                 | Params
------------------------------------------------------
0 | q_net        | NNLunarLanderDueling | 18.3 K
1 | target_q_net | NNLunarLanderDueling | 18.3 K
------------------------------------------------------
36.6 K    Trainable params
0         Non-trainable params
36.6 K    Total params
0.146     Total estimated model params size (MB)
/Users/meinczinger/.pyenv/versions/3.10.9/envs/rl/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

![DQN](dqn_average_return.png)